In [29]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
main_page = "https://hentaiz.bot/"


In [3]:
regex = 'data-source="https://e.streamqq.com/.*"'

def pull_link_from_text(text:str) -> list[str]:
    matches = re.search(regex, text).group()
    return [matches.removeprefix('data-source="').removesuffix('"')]

def pull_link(link:str) -> list[str]:
    response = requests.get(link)
    text = response.text
    return pull_link_from_text(text)

pull_link("https://hentaiz.bot/hentai/yoasobi-gurashi-1-67b5659807270")

['https://e.streamqq.com/videos/67b4abfe74a2d0cad1041e28/play?event_id=player-wrapper']

In [4]:
def get_page_link(page_num : int)-> str:
    return f"https://hentaiz.bot/?page={page_num}"
test_page = get_page_link(2)
print(test_page)
test_page = requests.get(test_page).text

https://hentaiz.bot/?page=2


In [5]:
'https://hentaiz.bot/hentai/' in test_page

True

In [11]:
def extract_items_with_views(text):
    soup = BeautifulSoup(text, 'html.parser')
    
    items = []
    
    for box in soup.select('div.item-box'):
        title_tag = box.select_one('a[title]')
        main_img_tag = box.select_one('img.image-box__poster__poster')
        episode_img_tag = box.select_one('img.image-box__poster__episode')
        views_tag = box.select_one('div.item-box__views')
        
        if title_tag and main_img_tag:
            title = title_tag['title']
            link = title_tag['href']
            main_img = main_img_tag['src']
            episode_img = episode_img_tag['src'] if episode_img_tag else None
            views = views_tag.get_text(strip=True) if views_tag else None
            
            # Remove non-digit characters from views and convert to int
            if views:
                views = int(''.join(filter(str.isdigit, views)))
            
            items.append({
                'title': title,
                'link': link,
                'main_image': main_img,
                'episode_image': episode_img,
                'views': views
            })
            
    return items

In [12]:
extract_items_with_views(test_page)


[{'title': 'Tsuma netori rei boku no ayamachi kanojo no sentaku 1',
  'link': 'https://hentaiz.bot/hentai/tsuma-netori-rei-boku-no-ayamachi-kanojo-no-sentaku-1',
  'main_image': 'https://hentaiz.bot/img/300/2024/11/13/2f09d84bc6aee58aef5873731f007f36ac2e60f32f53f115fe10459ad3a6107c.jpg',
  'episode_image': 'https://hentaiz.bot/static/hentaiz/images/episodes/1.webp',
  'views': 161420},
 {'title': 'Incha couple ga yougal-tachi to sex training suru hanashi 1',
  'link': 'https://hentaiz.bot/hentai/incha-couple-ga-yougal-tachi-to-sex-training-suru-hanashi-1',
  'main_image': 'https://hentaiz.bot/img/300/2024/10/31/39675ffff38eb40877b6dbfceb51a91a65f11b016abeb8796c26f448e7a87d05.jpg',
  'episode_image': 'https://hentaiz.bot/static/hentaiz/images/episodes/1.webp',
  'views': 154124},
 {'title': 'Seika jogakuin kounin sao ojisan 6',
  'link': 'https://hentaiz.bot/hentai/seika-jogakuin-kounin-sao-ojisan-6',
  'main_image': 'https://hentaiz.bot/img/300/2024/12/05/aef909c414b6f28dccc85d5c9ad5b6

In [33]:
#crawl page links
max_page = 111
exception = []
video_get_exception = []
vids : list[dict] = []
for page in range(max_page):
    page_text = requests.get(get_page_link(page)).text
    try:
        items = extract_items_with_views(page_text)
        for vid_page in items:
            page_text = requests.get(vid_page['link']).text
            try:
                item =  pull_link_from_text(page_text)[0]
                vid_page["video_link"] = item
                
            except:
                video_get_exception.append(vid_page)
        vids += items
        if page % 15 == 0:
            files = open("raw_data.json", 'w')
            json.dump(vids,files)
            files.close()
            print(f"Done {page} page")
    except:
        exception.append(page)
files = open("raw_data.json", 'w')
json.dump(vids,files)
files.close()
print(exception)

Done 0 page
Done 15 page
Done 30 page
Done 45 page
Done 60 page
Done 75 page
Done 90 page
Done 105 page
[]


In [34]:
vids

[{'title': 'Kyokugen chikan tokuiten 3 the animation 1',
  'link': 'https://hentaiz.bot/hentai/kyokugen-chikan-tokuiten-3-the-animation-1',
  'main_image': 'https://hentaiz.bot/img/300/2024/11/08/40b914ca1a370a41199547d0c7e76e86efaf2368e1b5e2f7d4f5d509557ecd07.jpg',
  'episode_image': 'https://hentaiz.bot/static/hentaiz/images/episodes/1.webp',
  'views': 89129,
  'video_link': 'https://e.streamqq.com/videos/672e31c72f0b2550e308077e/play?event_id=player-wrapper'},
 {'title': 'Taishou itsuwari bridal migawari hanayome to gunpuku no mou ai 1',
  'link': 'https://hentaiz.bot/hentai/taishou-itsuwari-bridal-migawari-hanayome-to-gunpuku-no-mou-ai-1',
  'main_image': 'https://hentaiz.bot/img/300/2025/08/31/4ea20a514500c2404d3038fe2e4c8bdf7f334bc24f869c487c924f01e2a2de78.jpg',
  'episode_image': 'https://hentaiz.bot/static/hentaiz/images/episodes/1.webp',
  'views': 30179,
  'video_link': 'https://e.streamqq.com/videos/68b525c683911575000132f2/play?event_id=player-wrapper'},
 {'title': 'Iribit

In [43]:
import sqlite3
import json
conn = sqlite3.connect('videosss_db.db')
cursor = conn.cursor()




# Create the videos table if it doesn't exist
# Using AUTOINCREMENT for a unique, auto-increasing primary key
cursor.execute('''
CREATE TABLE IF NOT EXISTS videos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    link TEXT NOT NULL,
    title TEXT NOT NULL,
    image_url TEXT,
    view INTEGER NOT NULL
)
''')

# Clear existing data to prevent duplicates on re-running the script
cursor.execute('DELETE FROM videos')

# Insert the data into the database
# We'll pair each video with an image. If lists are different lengths,
# this will only pair up to the length of the shorter list.
for vid in vids:

    
    # Using parameterized queries to prevent SQL injection
    cursor.execute(
        "INSERT INTO videos (link, title, image_url,view) VALUES (?, ?, ?,?)",
        (vid['video_link'],vid['title'] , vid['main_image'], int(vid['views']))
    )

# Commit the changes to the database and close the connection
conn.commit()
conn.close()

print("Database 'videos.db' has been successfully created and populated.")



Database 'videos.db' has been successfully created and populated.
